<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/World_Health_Organization_Logo.svg/1280px-World_Health_Organization_Logo.svg.png" alt="image info" />

# Train-test Split, Feature Engineering, & Modelling

## 0. Imports & Loading Data

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import root_mean_squared_error

def load_data(file="Life Expectancy Data.csv"):
    df = pd.read_csv(file)
    return df

## 1. Train-test Splitting

We use this step to divide the dataset in **train** and **test** subsets - allowing us to train the model on one portion of the data and evaluate its performance on unseen data (test set)  
Our target variable is _Life Expectancy_, all other columns are used as features  

We apply a 80/20 split - this is an industry standard that provides enough data for model training while keeping a fair sample aside for valuation.  
We selected a random state of **42** so that we can compare iterations of the model, this should be removed for production  

After splitting we check the indices and lengths match correctly - confirming the split worked as expected.


In [16]:
def split_data(df, target="Life_expectancy"):
    X = df.drop(columns=target)
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    print("Test Train-test Splitting")
    print(f"Matching lengths: {len(X_train) == len(y_train)}")
    print(f"Index mismatch count: {sum(X_train.index != y_train.index)}")
    return X_train, X_test, y_train, y_test

## 2. Feature Engineering

In this step we refine our input data to make it more informative, interpretable, and suitable for modelling


### 2.1 Dropping Irrelevant Features

- Features like _Country_ & _Year_ do not carry any predictive information
  - _Country_ acts more like an identifier
  - _Year_ represents a time index
- We also remove the Population in millions (_Population_m1n_) since EDA showed it had little correlation with Life Expectancy (< 0.4)

In [17]:
def drop_irrelevant_features(X_train, X_test):
    drop_cols = ['Country', 'Year', 'Population_mln']
    return X_train.drop(columns=drop_cols), X_test.drop(columns=drop_cols)

### 2.2 Dropping Collinear Variables

Collinear variables are when two or more features are strongly correlated with each other  
- _Thinness five to nine years_ and _Thinness ten to nineteen years_ are strongly collinear so we drop _Thinness five to nine years_ as it has a slightly worse correlation
- *Economy_status_Developing* and *Economy_status_Developed* are one hot encoded so we drop the *Economy_status_Developing* column as all the information is stored in *Economy_status_Developed*  

This helps prevent instability in the model and improves interpretability


In [18]:
def drop_collinear_features(X_train, X_test):
    drop_cols = ['Thinness_five_nine_years', 'Economy_status_Developing']
    return X_train.drop(columns=drop_cols), X_test.drop(columns=drop_cols)

### 2.3 Log Transformations

Log transformations correct **non-linear relationships**, these were identified during our EDA:
- *GDP_per capita*
- *BMI*
- *Under_five_deaths*

We drop GDP_per_capita and BMI to avoid duplication, this improves model accuracy and numerical   
We decided to keep Under_five_deaths as we saw there was still a linear correlation at low values to Life Expectancy

In [19]:
def log_transform_features(X_train, X_test):
    log_vars = {
        'GDP_per_capita': 'Log_GDP',
        'BMI': 'Log_BMI',
        'Under_five_deaths': 'Log_under_five_deaths'
    }
    for col, new_col in log_vars.items():
        X_train[new_col] = np.log(X_train[col])
        X_test[new_col] = np.log(X_test[col])

    # Drop only GDP and BMI, keep Under_five_deaths
    X_train = X_train.drop(columns=['GDP_per_capita', 'BMI'])
    X_test  = X_test.drop(columns=['GDP_per_capita', 'BMI'])
    return X_train, X_test

### 2.4 One-Hot Encoding: Region

We one-hot encode *Region* to capture **geographical, cultural and political effects** that impact life expectancy  

**[The Lancet](https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(16)31012-1/fulltext)** has an article that touches on some of these topics


In [20]:
def one_hot_encode_region(X_train, X_test):
    X_train = pd.get_dummies(X_train, columns=['Region'], drop_first=True, dtype=int)
    X_test = pd.get_dummies(X_test, columns=['Region'], drop_first=True, dtype=int)
    return X_train, X_test

### 2.5 Combining Disease Variables

We noticed _Hepatitis B_, _Polio_, and _Diphtheria_ were highly correlated, as they represent similar vaccination metrics  
To simplify and stabilize the model, we averaged (mean) these into a single composite feature: _Disease_

This technique is known as **feature aggregation**. This is combining variables that represent similar predictive power to the model into one.  
This will both retain their ability to map life expectancy, while stabilising the features against multicollinearity.  
This improves our model and makes it reproducible and robust.


In [21]:
def combine_disease_features(X_train, X_test):
    cols = ['Hepatitis_B', 'Polio', 'Diphtheria']
    X_train['Disease'] = X_train[cols].mean(axis=1)
    X_test['Disease'] = X_test[cols].mean(axis=1)
    X_train = X_train.drop(columns=cols)
    X_test = X_test.drop(columns=cols)
    return X_train, X_test

### 2.6 Scaling

We scale our numeric variables using _Robust Scaling_  
Scaling helps ensure all features contribute equally to the model, and we found it helped with multicollinearity issues by reducing the absolute relationship between potentially non correlated variables.

We chose _RobustScaler_ because it's resistant to outliers - and our data contains several extreme values, particularly in mortality and disease incidence  
Furthermore the data is not normally distributed, so we could not use a _Standard Scaler_

This step improves the model's _condition number_, which measures numerical stability


In [22]:
def robust_scale(X_train, X_test, y_train):
    cols = list(X_train.columns)

    # Exclude non-numerical columns
    excluded_cols = [
        'Economy_status_Developed', 'Economy_status_Developing', 'Region_Asia',
        'Region_Central America and Caribbean', 'Region_European Union', 'Region_Middle East',
        'Region_North America', 'Region_Oceania', 'Region_Rest of Europe', 'Region_South America'
    ]
    scale_cols = [x for x in cols if x not in excluded_cols]

    scaler = RobustScaler()
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    X_train_scaled[scale_cols] = scaler.fit_transform(X_train[scale_cols])
    X_test_scaled[scale_cols] = scaler.transform(X_test[scale_cols])

    X_train_const = sm.add_constant(X_train_scaled)
    X_test_const = sm.add_constant(X_test_scaled)

    print("\nTest Scaling")
    numeric_check = all(np.issubdtype(dtype, np.number) for dtype in X_train_const.dtypes)
    print(f"All columns numeric: {numeric_check}")
    null_count = X_train_const.isnull().sum().sum()
    print(f"Null values in training set: {null_count}")
    index_match = all(X_train_const.index == y_train.index)
    print(f"Index alignment with y_train: {index_match}")

    return X_train_const, X_test_const, scaler

In [23]:
# Build function with all Feature Engineering included
def feature_engineering_pipeline(X_train, X_test, y_train):
    X_train, X_test = drop_irrelevant_features(X_train, X_test)
    X_train, X_test = drop_collinear_features(X_train, X_test)
    X_train, X_test = log_transform_features(X_train, X_test)
    X_train, X_test = one_hot_encode_region(X_train, X_test)
    X_train, X_test = combine_disease_features(X_train, X_test)
    X_train, X_test, scaler = robust_scale(X_train, X_test, y_train)
    return X_train, X_test, scaler

## 3. Feature Selection

### 3.1 Lasso Regression

Lasso regressions penalize regressions with too many predictors.  
They reduce the variables with little affect on the model.  
This is done by using an L1 penalty, absolutely shrinking all coeff


In [24]:
def lasso_feature_selection(X_train_const, y_train):
    print("\nLasso:")
    lasso = LassoCV(cv=5, random_state=0)
    lasso.fit(X_train_const, y_train)
    selected_features = X_train_const.columns[lasso.coef_ != 0]
    print("Remaining variable:")
    print(selected_features)
    return selected_features

### 3.2 Reducing Multicollinearity (VIF)

Variance Inflation Factor (VIF) identifies variables causing multicollinearity (VIF > 5 is problematic).

> This a piece of code from stats.stackexchange.com

In [25]:
def calculate_vif(X, thresh=10):
    print("\nVIF:")
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]
        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True
    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

## 4. Modelling

In [26]:
def train_ols_model(X_train, y_train, X_test, y_test):
    model = sm.OLS(y_train, X_train).fit()
    display(model.summary())

    y_pred_train = model.predict(X_train)
    y_pred = model.predict(X_test)

    rmse_t = np.sqrt(root_mean_squared_error(y_train, y_pred_train))
    rmse_e = np.sqrt(root_mean_squared_error(y_test, y_pred))
    print(f"Train RMSE: {rmse_t:.3f}")
    print(f"Test RMSE: {rmse_e:.3f}")

    print(f"R-squared: {model.rsquared:.3f}")
    print(f"Condition Number: {model.condition_number:.3f}")
    return model


### 4.1 Main Predictive Model

In [27]:
# Load data
df = load_data("Life Expectancy Data.csv")

# Train-test split
X_train_raw, X_test_raw, y_train, y_test = split_data(df)

# Feature engineering pipeline
X_train_feature_eng, X_test_feature_eng, scaler = feature_engineering_pipeline(X_train_raw, X_test_raw, y_train)

# Lasso feature selection
selected_features = lasso_feature_selection(X_train_feature_eng, y_train)

# Select features + constant automatically included
X_train_selected = X_train_feature_eng[['const'] + [f for f in selected_features if f != 'const']]
X_test_selected = X_test_feature_eng[['const'] + [f for f in selected_features if f != 'const']]

# Reduce multicollinearity using VIF
X_train_final = calculate_vif(X_train_selected)
X_test_final = X_test_selected[X_train_final.columns]  # align test set columns with training set

# Train OLS model
final_model = train_ols_model(X_train_final, y_train, X_test_final, y_test)

Test Train-test Splitting
Matching lengths: True
Index mismatch count: 0

Test Scaling
All columns numeric: True
Null values in training set: 0
Index alignment with y_train: True

Lasso:
Remaining variable:
Index(['Under_five_deaths', 'Adult_mortality', 'Measles', 'Incidents_HIV',
       'Schooling', 'Economy_status_Developed', 'Log_GDP', 'Log_BMI',
       'Log_under_five_deaths', 'Region_Central America and Caribbean',
       'Region_Oceania', 'Region_South America'],
      dtype='object')

VIF:
dropping 'Log_under_five_deaths' at index: 9
Remaining variables:
Index(['const', 'Under_five_deaths', 'Adult_mortality', 'Measles',
       'Incidents_HIV', 'Schooling', 'Economy_status_Developed', 'Log_GDP',
       'Log_BMI', 'Region_Central America and Caribbean', 'Region_Oceania',
       'Region_South America'],
      dtype='object')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                 1.196e+04
Date:                Mon, 27 Oct 2025   Prob (F-statistic):               0.00
Time:                        12:58:05   Log-Likelihood:                -3737.2
No. Observations:                2291   AIC:                             7498.
Df Residuals:                    2279   BIC:                             7567.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   70.9191      0.045   1576.634      0.000      70.831      71.007
Under_five_deaths                       -4.2765      0.080    -53.230      0.000      -4.434      -4.119
Adult_mortality                         -6.7703      0.089    -76.489      0.000      -6.944      -6.597
Measles                                  0.0199      0.050      0.401      0.689      -0.078       0.117
Incidents_HIV                            0.0315      0.007      4.200      0.000       0.017       0.046
Schooling                                0.7762      0.087      8.874      0.000       0.605       0.948
Economy_status_Developed                 1.8788      0.098     19.086      0.000       1.686       2.072
Log_GDP                                  0.9612      0.087     11.039      0.000       0.790       1.132
Log_BMI                                 -0.5318      0.062     -8.568      0.000      -0.653      -0.410
Region_Central America and Caribbean     1.7073      0.095     17.893      0.000       1.520       1.894
Region_Oceania                          -0.8333      0.118     -7.055      0.000      -1.065      -0.602
Region_South America                     1.3974      0.109     12.771      0.000       1.183       1.612
==============================================================================
Omnibus:                       16.572   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.939
Skew:                           0.127   Prob(JB):                     4.68e-05
Kurtosis:                       3.380   Cond. No.                         33.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Train RMSE: 1.112
Test RMSE: 1.122
R-squared: 0.983
Condition Number: 33.431


### 4.2 Ethical Model

The full model includes a broader range of health-related features such as *Incidents_HIV*, *Measles*, and mortality rates (*Under_five_deaths* and *Adult_mortality*). While these improve predictive accuracy, they depend on data that reflect potential population-level health vulnerabilities, disease prevalence, and healthcare access. Even when aggregated, such statistics can be considered sensitive because they may influence how countries are perceived, not just in public health, but areas such as tourism, trade, and international reputation.

To limit these concerns, the simplified model removes all disease-specific statistics (*Incidents_HIV* and *Measles*) as well as Mortality rate features (*Infant_deaths*, *Under_five_deaths* and *Adult_mortality*). Therefore, the model keeps only general, non-medical factors such as average *BMI*, *GDP* per capita, *Schooling*, and *Region*. These variables describe living standards and development context rather than direct health performance, reducing the ethical risks associated with using potentially sensitive health data. 

Although this simplification reduces predictive accuracy, it strengthens important principles such as ethical transparency, fairness, and data minimisation. In summary, the simplified model is designed to provide reasonable life-expectancy estimates without relying on disease-related or mortality data, ensuring the tool remains transparent, interpretable, and socially responsible while still delivering valuable insights.

In [30]:
# Load data
df = load_data("Life Expectancy Data.csv")

# Train-test split
X_train_raw, X_test_raw, y_train, y_test = split_data(df)

# Feature engineering pipeline
X_train_feature_eng, X_test_feature_eng, scaler = feature_engineering_pipeline(X_train_raw, X_test_raw, y_train)

# Remove non-ethical columns BEFORE feature selection
non_ethical_cols = [
    'Infant_deaths', 'Under_five_deaths', 'Adult_mortality',
    'Alcohol_consumption', 'Measles', 'Incidents_HIV',
    'Thinness_ten_nineteen_years', 'Log_under_five_deaths', 'Disease'
]

X_train_ethical = X_train_feature_eng.drop(
    columns=[c for c in non_ethical_cols if c in X_train_feature_eng.columns], errors='ignore'
)
X_test_ethical = X_test_feature_eng.drop(
    columns=[c for c in non_ethical_cols if c in X_test_feature_eng.columns], errors='ignore'
)

# Lasso feature selection
selected_features_ethical = lasso_feature_selection(X_train_ethical, y_train)

# Select features + constant automatically included
X_train_selected_ethical = X_train_ethical[['const'] + [f for f in selected_features_ethical if f != 'const']]
X_test_selected_ethical  = X_test_ethical[['const'] + [f for f in selected_features_ethical if f != 'const']]

# Reduce multicollinearity using VIF
X_train_final_ethical = calculate_vif(X_train_selected_ethical)
X_test_final_ethical  = X_test_selected_ethical[X_train_final_ethical.columns]  # align test set columns with training set

# Train OLS model
ethical_model = train_ols_model(X_train_final_ethical, y_train, X_test_final_ethical, y_test)


Test Train-test Splitting
Matching lengths: True
Index mismatch count: 0

Test Scaling
All columns numeric: True
Null values in training set: 0
Index alignment with y_train: True

Lasso:
Remaining variable:
Index(['Schooling', 'Economy_status_Developed', 'Log_GDP', 'Log_BMI',
       'Region_Asia', 'Region_Central America and Caribbean',
       'Region_European Union', 'Region_Middle East', 'Region_North America',
       'Region_Oceania', 'Region_Rest of Europe', 'Region_South America'],
      dtype='object')

VIF:
Remaining variables:
Index(['const', 'Schooling', 'Economy_status_Developed', 'Log_GDP', 'Log_BMI',
       'Region_Asia', 'Region_Central America and Caribbean',
       'Region_European Union', 'Region_Middle East', 'Region_North America',
       'Region_Oceania', 'Region_Rest of Europe', 'Region_South America'],
      dtype='object')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.782
Method:                 Least Squares   F-statistic:                     686.4
Date:                Mon, 27 Oct 2025   Prob (F-statistic):               0.00
Time:                        14:45:10   Log-Likelihood:                -6650.5
No. Observations:                2291   AIC:                         1.333e+04
Df Residuals:                    2278   BIC:                         1.340e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   62.5700      0.240    261.174      0.000      62.100      63.040
Schooling                                0.4332      0.324      1.335      0.182      -0.203       1.069
Economy_status_Developed                 2.3882      0.558      4.280      0.000       1.294       3.483
Log_GDP                                  6.5311      0.288     22.700      0.000       5.967       7.095
Log_BMI                                  1.3559      0.248      5.468      0.000       0.870       1.842
Region_Asia                              9.1652      0.329     27.876      0.000       8.520       9.810
Region_Central America and Caribbean     8.7169      0.394     22.111      0.000       7.944       9.490
Region_European Union                    7.4275      0.580     12.801      0.000       6.290       8.565
Region_Middle East                       7.7692      0.454     17.124      0.000       6.879       8.659
Region_North America                     7.3873      0.815      9.063      0.000       5.789       8.986
Region_Oceania                           5.9455      0.482     12.333      0.000       5.000       6.891
Region_Rest of Europe                    9.1373      0.452     20.217      0.000       8.251      10.024
Region_South America                     8.8346      0.436     20.278      0.000       7.980       9.689
==============================================================================
Omnibus:                      100.026   Durbin-Watson:                   2.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              255.341
Skew:                          -0.212   Prob(JB):                     3.58e-56
Kurtosis:                       4.580   Cond. No.                         12.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Train RMSE: 2.100
Test RMSE: 2.142
R-squared: 0.783
Condition Number: 12.080


## 5. Model Performance & Notes

| Model            | Train RMSE | Test RMSE | R-squared | Condition Number |
|-----------------|------------|-----------|-----------|-----------------|
| Main Predictive  | 1.112      | 1.122     | 0.983     | 33.431          |
| Ethical          | 2.100      | 2.142     | 0.783     | 12.080          |

### 5.1 Robustness
When making sure a linear regression model is robust, we have focused on four main points:  
* Outliers  
* Collinearity  
* Over-fitting  
* Data leakage  



#### i. Outliers
We handled outliers using **Robust Scaling**, which reduces the effect of extreme values. This ensures the outliers do not dominate the scale, providing a more accurate representation for modelling. Robust scaling also helps **Lasso regression** identify irrelevant features correctly, aiding in over-fitting prevention.

#### ii. Collinearity
Collinearity was addressed through several steps:  
1. **Manual feature engineering**: dropping highly correlated features.  
2. **VIF analysis**: features with VIF > 10 were dropped.  

Scaling also ensured features on different scales weren’t misidentified as collinear. These steps stabilize the model and improve predictive robustness.

#### iii. Over-fitting
Over-fitting was mitigated by:  
* Dropping features like `Population` identified via EDA.  
* Applying **Lasso regression**, which shrinks coefficients and sets irrelevant ones to zero.  

Scaling ensures each variable’s proportional effect is considered, improving feature selection and enabling the model to generalize well.

#### iv. Data leakage
To prevent data leakage:  
* Transformations were applied **before testing**, with no back-and-forth adjustments. 
* Transformations were applied **after train-test splitting**, so no test data leaking into training data.   
* Scaling was **fit only on training data** and applied to test data.  
* Lasso regression was performed solely on training data to penalize feature counts, avoiding test information influencing selection.  

These steps make the model robust and safe for deployment.



### 5.2 Transparency

#### Pitfalls of Linear Regression
Linear regression assumes a **purely linear relationship** between predictors and target. While it provides overall insight, it may **miss complex relationships** both globally and per country. Accounting for these would risk over-fitting, so the model focuses on general trends rather than intricate local patterns.


# 6. Demo

> https://www.danjmbacon.co.uk/who